In [1]:
# IMPORTAMOS LIBRERIAS
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import mysql.connector

pd.set_option('display.max_columns', None) 


In [2]:
# DEFINIR LA ESTRUCTURA DE LA BBDD

In [3]:
# Crear la base de datos
cnx = mysql.connector.connect(
    user="root",
    password="AlumnaAdalab",
    host="127.0.0.1",
    auth_plugin='mysql_native_password'
)

cursor = cnx.cursor()
query_create_bd = "CREATE SCHEMA IF NOT EXISTS `ABCCorporation (M3T3)`"

cursor.execute(query_create_bd)

In [4]:

# Crear Tablas

# Table `ABCCorporation (M3T3)`.`PERSONAL_INFO`
# -----------------------------------------------------

query_create_table_personal = """
CREATE TABLE IF NOT EXISTS `ABCCorporation (M3T3)`.`PERSONAL_INFO` (
  `IdEmployee` BIGINT NOT NULL,
  `EmployeeNumber` VARCHAR(45) NULL,
  `Age` BIGINT NULL,
  `DateBirth` BIGINT NULL,
  `Gender` VARCHAR(45) NULL,
  `MaritalStatus` VARCHAR(45) NULL,
  `DistanceFromHome` BIGINT NULL,
  `Education` BIGINT NULL,
  `EducationField` VARCHAR(100) NULL,
  PRIMARY KEY (`IdEmployee`))
"""
cursor.execute(query_create_table_personal)


# Table `ABCCorporation (M3T3)`.`EMPLOYEE_SATISFACTION`
# ----------------------------------------

query_create_table_satisfaction = """
CREATE TABLE IF NOT EXISTS `ABCCorporation (M3T3)`.`EMPLOYEE_SATISFACTION` (
  `IdEmployeeSatisfaction` INT NOT NULL AUTO_INCREMENT,
  `EnvironmentSatisfaction` INT NULL,
  `JobInvolvement` INT NULL,
  `JobSatisfaction` INT NULL,
  `RelationshipSatisfaction` INT NULL,
  `WorkLifeBalance` INT NULL,
  PRIMARY KEY (`IdEmployeeSatisfaction`))

"""

cursor.execute(query_create_table_satisfaction)


# Table `ABCCorporation (M3T3)`.`JOB_DATA`
# -----------------------------------------------------

query_create_table_job = """
CREATE TABLE IF NOT EXISTS `ABCCorporation (M3T3)`.`JOB_DATA` (
  `IdJobData` INT NOT NULL AUTO_INCREMENT,
  `Attrition` VARCHAR(45) NULL,
  `JobLevel` INT NULL,
  `JobRole` VARCHAR(100) NULL,
  `NumCompaniesWorked` INT NULL,
  `TotalWorkingYears` INT NULL,
  `YearsAtCompany` INT NULL,
  `YearsSinceLastPromotion` INT NULL,
  `YearsWithCurrManager` INT NULL,
  PRIMARY KEY (`IdJobData`))
"""

cursor.execute(query_create_table_job)


# Table `ABCCorporation (M3T3)`.`SALARY`
# -----------------------------------------------------

query_create_table_salary = """
CREATE TABLE IF NOT EXISTS `ABCCorporation (M3T3)`.`SALARY` (
  `IdSalary` INT NOT NULL AUTO_INCREMENT,
  `HourlyRate` FLOAT NULL,
  `DailyRate` FLOAT NULL,
  `MonthlyRate` FLOAT NULL,
  `PercentSalaryHike` INT NULL,
  PRIMARY KEY (`IdSalary`))
  
"""

cursor.execute(query_create_table_salary)

# Table `ABCCorporation (M3T3)`.`JOB_EXTRAS`
# -----------------------------------------------------

query_create_table_extras = """
CREATE TABLE IF NOT EXISTS `ABCCorporation (M3T3)`.`JOB_EXTRAS` (
  `IdJobExtras` INT NOT NULL AUTO_INCREMENT,
  `OverTime` VARCHAR(45) NULL,
  `BusinessTravel` VARCHAR(45) NULL,
  `StockOptionLevel` INT NULL,
  `TrainingTimesLastYear` INT NULL,
  `RemoteWork` TINYINT NULL,
  `PerformanceRating` INT NULL,
  PRIMARY KEY (`IdJobExtras`))

"""

cursor.execute(query_create_table_extras)

# Table `ABCCorporation (M3T3)`.`UNION`
# -----------------------------------------------------

query_create_table_union = """
CREATE TABLE IF NOT EXISTS `ABCCorporation (M3T3)`.`UNION` (
  `IdUnion` INT NOT NULL AUTO_INCREMENT,
  `PERSONAL_INFO_IdEmployee` BIGINT NOT NULL,
  `EMPLOYEE_SATISFACTION_IdEmployeeSatisfaction` INT NOT NULL,
  `JOB_DATA_IdJobData` INT NOT NULL,
  `SALARY_IdSalary` INT NOT NULL,
  `JOB_EXTRAS_IdJobExtras` INT NOT NULL,
  PRIMARY KEY (`IdUnion`, `PERSONAL_INFO_IdEmployee`, `EMPLOYEE_SATISFACTION_IdEmployeeSatisfaction`, `JOB_DATA_IdJobData`, `SALARY_IdSalary`, `JOB_EXTRAS_IdJobExtras`),
  INDEX `fk_UNION_PERSONAL_INFO_idx` (`PERSONAL_INFO_IdEmployee` ASC) VISIBLE,
  INDEX `fk_UNION_EMPLOYEE_SATISFACTION1_idx` (`EMPLOYEE_SATISFACTION_IdEmployeeSatisfaction` ASC) VISIBLE,
  INDEX `fk_UNION_JOB_DATA1_idx` (`JOB_DATA_IdJobData` ASC) VISIBLE,
  INDEX `fk_UNION_SALARY1_idx` (`SALARY_IdSalary` ASC) VISIBLE,
  INDEX `fk_UNION_JOB_EXTRAS1_idx` (`JOB_EXTRAS_IdJobExtras` ASC) VISIBLE,
  CONSTRAINT `fk_UNION_PERSONAL_INFO`
    FOREIGN KEY (`PERSONAL_INFO_IdEmployee`)
    REFERENCES `ABCCorporation (M3T3)`.`PERSONAL_INFO` (`IdEmployee`)
    ON DELETE NO ACTION
    ON UPDATE NO ACTION,
  CONSTRAINT `fk_UNION_EMPLOYEE_SATISFACTION1`
    FOREIGN KEY (`EMPLOYEE_SATISFACTION_IdEmployeeSatisfaction`)
    REFERENCES `ABCCorporation (M3T3)`.`EMPLOYEE_SATISFACTION` (`IdEmployeeSatisfaction`)
    ON DELETE NO ACTION
    ON UPDATE NO ACTION,
  CONSTRAINT `fk_UNION_JOB_DATA1`
    FOREIGN KEY (`JOB_DATA_IdJobData`)
    REFERENCES `ABCCorporation (M3T3)`.`JOB_DATA` (`IdJobData`)
    ON DELETE NO ACTION
    ON UPDATE NO ACTION,
  CONSTRAINT `fk_UNION_SALARY1`
    FOREIGN KEY (`SALARY_IdSalary`)
    REFERENCES `ABCCorporation (M3T3)`.`SALARY` (`IdSalary`)
    ON DELETE NO ACTION
    ON UPDATE NO ACTION,
  CONSTRAINT `fk_UNION_JOB_EXTRAS1`
    FOREIGN KEY (`JOB_EXTRAS_IdJobExtras`)
    REFERENCES `ABCCorporation (M3T3)`.`JOB_EXTRAS` (`IdJobExtras`)
    ON DELETE NO ACTION
    ON UPDATE NO ACTION)
"""

cursor.execute(query_create_table_union)

In [5]:
# Leer archivo CSV en un DataFrame
df = pd.read_csv('files/df_fase1.csv')


In [6]:
# Crear la conexión con la base de datos usando SQLAlchemy

cnx = mysql.connector.connect(
    user="root",
    password="AlumnaAdalab",
    host="127.0.0.1",
    auth_plugin='mysql_native_password'
)

cursor = cnx.cursor()

In [7]:
# Funcion para convertir a entero
def convertir_int64_a_int(df): 
    
    # Convertir solo las columnas de tipo int64 
    for col in df.select_dtypes(include=['int64', 'int32', 'int']).columns: 
        df[col] = df[col].astype(int)
    
    return df
    


In [8]:
df_int = convertir_int64_a_int(df)

df_int.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1501 entries, 0 to 1500
Data columns (total 33 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                1501 non-null   int32  
 1   IdEmployee                1501 non-null   int32  
 2   EmployeeNumber            1501 non-null   object 
 3   Age                       1501 non-null   int32  
 4   DateBirth                 1501 non-null   int32  
 5   Attrition                 1501 non-null   object 
 6   BusinessTravel            1501 non-null   object 
 7   DailyRate                 1501 non-null   float64
 8   DistanceFromHome          1501 non-null   int32  
 9   Education                 1501 non-null   int32  
 10  EducationField            1501 non-null   object 
 11  EnvironmentSatisfaction   1501 non-null   int32  
 12  Gender                    1501 non-null   object 
 13  HourlyRate                1501 non-null   float64
 14  JobInvol

In [9]:
# Funcion para convertir a entero
def convertir(x): 
    
    # Convertir solo las columnas de tipo int64 
    lista = []
 
    for tupla in x:
        lista_intermedia = []
    
        for elem in tupla:
            try:
                lista_intermedia.append(int(elem))
            except:
                lista_intermedia.append(elem)
        
        lista.append(tuple(lista_intermedia))
        
    return lista

In [11]:
datos_tabla_personal = list(set(zip
                                (df["IdEmployee"].values, 
                                 df["EmployeeNumber"].values,
                                 df["Age"].values,
                                 df["DateBirth"].values,
                                 df["Gender"].values,
                                 df["MaritalStatus"].values,
                                 df["DistanceFromHome"].values,
                                 df["Education"].values,
                                 df["EducationField"].values,
                                 )))

In [12]:
datos_personal = convertir(datos_tabla_personal)

datos_personal

[(213, '329,0', 53, 1971, 'Male', 'no-data', 2, 4, 'Life Sciences'),
 (435, '878,0', 37, 1987, 'Male', 'no-data', 8, 3, 'Life Sciences'),
 (1159, 'no-data', 34, 1990, 'Female', 'no-data', 2, 2, 'no-data'),
 (712, '902,0', 44, 1980, 'Female', 'Married', 1, 3, 'Life Sciences'),
 (685, '705,0', 24, 2000, 'Male', 'no-data', 4, 3, 'no-data'),
 (492, '230,0', 39, 1985, 'Female', 'Single', 1, 4, 'Life Sciences'),
 (745, '1161,0', 38, 1986, 'Female', 'no-data', 25, 2, 'no-data'),
 (630, '337,0', 32, 1992, 'Male', 'Divorced', 3, 4, 'Medical'),
 (732, '1084,0', 31, 1993, 'Male', 'Married', 20, 3, 'Other'),
 (1194, 'no-data', 53, 1971, 'Male', 'Single', 5, 3, 'no-data'),
 (671, '593,0', 23, 2001, 'Male', 'Single', 25, 1, 'Technical Degree'),
 (342, '1934,0', 48, 1976, 'Male', 'Married', 14, 4, 'no-data'),
 (261, '927,0', 42, 1982, 'Female', 'Divorced', 2, 4, 'Life Sciences'),
 (430, '577,0', 36, 1988, 'Male', 'Single', 22, 3, 'Life Sciences'),
 (624, '288,0', 47, 1977, 'Male', 'no-data', 1, 4, 'n

In [13]:
# Insertar datos

query_insertar_tablapersonal = """ INSERT INTO `ABCCorporation (M3T3)`.`PERSONAL_INFO` (`IdEmployee`, `EmployeeNumber`, `Age`, `DateBirth`, `Gender`, `MaritalStatus`, `DistanceFromHome`, `Education`, `EducationField`) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"""

datos_tabla_personal = list(set(zip
                                (df["IdEmployee"].values, 
                                 df["EmployeeNumber"].values,
                                 df["Age"].values,
                                 df["DateBirth"].values,
                                 df["Gender"].values,
                                 df["MaritalStatus"].values,
                                 df["DistanceFromHome"].values,
                                 df["Education"].values,
                                 df["EducationField"].values,
                                 )))


cursor.executemany(query_insertar_tablapersonal, datos_personal)

cnx.commit()
